In [1]:
import sys
import os
import numpy as np

path = 'FacialLandmarks/BU4DFE_BND_V1.1/'
X=[]
Y=[]
# 101 faces - sub folders, 6 emotions - Angry, Disgust, Fear, Happy, Sad, Surprise
for face_dir in os.listdir(path)[1:]:
    face_path = path+face_dir+'/'
    for label in os.listdir(face_path):
        emotion_path = face_path+label+'/'
        for file in os.listdir(emotion_path):
            if file.endswith(".bnd") or file.endswith(".landmark"):
                file_path = emotion_path+file
                points = np.loadtxt(emotion_path+file, usecols=(1, 2, 3), encoding='utf-8') #x,y,z
                X.append(points)
                Y.append(label)
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape[0], -1)
x_length = len(X)
y_length = len(Y)


In [2]:
from math import acos
import math

pi=round(2*acos(0.0), 3)

def translate_data(X):
    X_translated =[]
    for face in X:
        mean = np.mean(face,axis=0)
        print(mean)
        translated_face = face - mean
        X_translated.append(translated_face)
    return X_translated

def rotate_data(X, axis='x'):
    sin_angle = math.sin(pi) #0
    cos_angle = math.cos(pi) #-1
    rotated_X=[]
    rotated_face = []
    if axis=='x':
        for face in X:
            for point in face:
                rotated_face.append([point[0], cos_angle * point[1] - sin_angle * point[2], sin_angle * point[1] + cos_angle * point[2]])
            rotated_X.append(rotated_face)
    if axis=='y':
        for face in X:
            for point in face:
                rotated_face.append([cos_angle * point[0] + sin_angle * point[2], point[1], -sin_angle * point[0] + cos_angle * point[2]])
            rotated_X.append(rotated_face)
    if axis=='z':
        for face in X:
            for point in face:
                rotated_face.append([cos_angle * point[0] - sin_angle * point[1], sin_angle * point[0] + cos_angle * point[1], point[2]])
            rotated_X.append(rotated_face)
    else:
        raise ValueError("Axis must be 'x', 'y', or 'z'")
    
    return rotated_X


In [3]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

def classification(X,Y, classifier_type='RF'):
    # Select the classifier
    if classifier_type == 'RF':
        print("RF running")
        clf = RandomForestClassifier()
    elif classifier_type == 'SVM':
        clf = SVC()
    elif classifier_type == 'TREE':
        clf = DecisionTreeClassifier()

  # Define the scoring metrics
    scoring = ['accuracy', 'precision_macro', 'recall_macro']
    Y_pred = []
    test_indices = []
    cv = StratifiedKFold(n_splits=10)
    for (train,test) in cv.split(X,Y):
        # print(train,test)
        # print(len(train),len(test))
        # print(X[train],Y[train])
        clf.fit(X[train],Y[train])
        Y_pred.append(clf.predict(X[test]))
        test_indices.append(test)
    
    return Y,Y_pred,test_indices
    # scores = cross_validate(clf, data, labels, cv=cv, scoring=scoring, return_train_score=False)

    # # Since cross_validate returns a dictionary, you can process it however you need.
    # # For example, to print the mean scores:
    # for score_name, score_values in scores.items():
    #     print(f"{score_name}: {np.mean(score_values)}")

    # return scores

In [4]:
y_original,y_prediction,testi = classification(np.array(X),np.array(Y),'RF')

RF running


In [5]:
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import cross_val_score

def PrintEvalMetrics(pred, indices, y):
    finalPredictions = []
    groundTruth = []
    for p in pred:
        finalPredictions.extend(p)
    for i in indices:
        groundTruth.extend(y[i])
    print(confusion_matrix(finalPredictions, groundTruth))
    print("Precision: ", precision_score(groundTruth, finalPredictions, average='macro'))
    print("Recall: ", recall_score(groundTruth, finalPredictions, average='macro'))
    print("Accuracy: " , accuracy_score(groundTruth, finalPredictions))

In [6]:
PrintEvalMetrics(y_prediction,testi,y_original)

[[4556 2183 1479 1625 3065  638]
 [1398 3372 1753  836 1103  811]
 [ 692 1230 1847 1290 1324  691]
 [ 524  882 1130 4081  509  369]
 [1335  657  791  519 2476  769]
 [1619 1847 3044 1622 1665 6780]]
Precision:  0.3821256707478118
Recall:  0.38214498243886913
Accuracy:  0.3819407720782655
